In [31]:
# Firstly, some necessary imports

# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
# Misc
import sys

# Basics of Python data handling and visualization
import numpy as np
# The core of this example
from eolearn.core import EOTask, LinearWorkflow, FeatureType, OverwritePermission, LoadFromDisk, SaveToDisk

# Basics of GIS
# Machine learning 

# Suppress warnings
sys.stderr = open(os.devnull, "w")

import os
os.getcwd()
os.chdir("C:\\Users\\benos\\OneDrive\\Documents\\IJS\\Perceptive-Sentinel\\eo-learn\\examples\\experiments")
os.getcwd()


import cv2
#def rgb2gray(rgb):
#    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

class Segmentation(EOTask):
    
    def __init__(self, 
                 edge_features,
                 excluded_features,
                 dilation_mask,
                 erosion_mask,
                 output_feature):
        
        self.edge_features = edge_features
        self.excluded_features = excluded_features
        self.dilation_mask = dilation_mask
        self.erosion_mask = erosion_mask
        self.output_feature = output_feature
                 
    def debug(self, object):
        print(type(object),object)
        '''
                base_bands_feature,
                ndvi_feature,
                feature_vegetation,
                feature_edges,
                dilation_mask,
                closing_mask,
                low_treshold,
                high_treshold,
                ndvi_treshold,
                join_edges_treshold,
                join_vegetation_treshold,
                feature_join):
        
        self.feature_vegetation = feature_vegetation
        self.feature_edges = feature_edges
        self.dilation_mask = dilation_mask
        self.closing_mask = closing_mask
        self.low_treshold = low_treshold
        self.high_treshold = high_treshold
        self.ndvi_treshold = ndvi_treshold
        self.join_edges_treshold = join_edges_treshold
        self.join_vegetation_treshold = join_vegetation_treshold
        
        self.f = self._parse_features(base_bands_feature, new_names=False)
        self.base_feature_parsed = list(FeatureParser._parse_features(base_bands_feature, new_names=False).items())[0]
        self.base_feature = (self.base_feature_parsed[0],list(self.base_feature_parsed[1].items())[0][0])

        self.ndvi_feature_parsed = list(FeatureParser._parse_features(ndvi_feature, new_names=False).items())[0]
        self.ndvi_feature = (self.ndvi_feature_parsed[0],list(self.ndvi_feature_parsed[1].items())[0][0])
        
        self.feature_join = feature_join
        '''
    def extract_edges(self, eopatch, feature_type, feature_name, feature_weight, low_threshold, high_threshold, blur):
        
        image = eopatch[feature_type][feature_name]
        dims = len(image.shape)
        #print(dims)
        #print(image.shape)
        if dims == 4:
            t, w, h, _ = image.shape
            sum_edges = np.zeros((w,h))
            all_edges = np.zeros((t,w,h))
            for time in range(t):
                image_one = image[time]
                edge =  self.one_edge(image_one, low_threshold, high_threshold, blur)
                #print(edge.shape)
                #print(all_edges[t].shape)
                all_edges[time] = edge
                sum_edges = sum_edges + edge
            eopatch.add_feature(FeatureType.MASK,feature_name+'_EDGE', all_edges[..., np.newaxis])
            sum_edges = sum_edges / t
            sum_edges = sum_edges > feature_weight
            eopatch.add_feature(FeatureType.MASK_TIMELESS,feature_name+'_EDGE', sum_edges[..., np.newaxis])
            return sum_edges
        else:
            w, h, _ = image.shape
            edges = self.one_edge(image, low_threshold, high_threshold, blur)
            eopatch.add_feature(FeatureType.MASK_TIMELESS, feature_name+'_EDGE', edges[..., np.newaxis]) 
            return edges
    
    def one_edge(self, image, low_threshold, high_threshold, blur):
            ##########QUICK NORMALIZATION -  SHOULD BE LATER IMPROVED / MOVED SOMEWHERE ELSE      
            f_min = np.min(image)
            f_max = np.max(image)
            image = (image-f_min)/f_max*255
            image = image.squeeze()
            ############
            kernel_size,sigma = blur
            smoothed_image = cv2.GaussianBlur(image, kernel_size, sigma)
            edges = cv2.Canny(smoothed_image.astype(np.uint8), low_threshold, high_threshold)
            return edges
    
    
    def filter_unwanted_areas(self, eopatch, feature, threshold, dilation, erosion):
        #Returns mask of areas that should be excluded (low NDVI etc...)
        bands = eopatch.data['BANDS']
        t, w, h, _ = bands.shape
        mask = np.zeros((w,h))
        
        for time in range(t):
            fet = eopatch[feature[0]][feature[1]][time].squeeze()
            mask_cur = fet<=threshold
            mask = np.logical_or(mask_cur, mask)    
        mask = cv2.dilate((mask_cur*255).astype(np.uint8), dilation*255)
        mask = cv2.erode((mask_cur*255).astype(np.uint8), erosion*255)
        #eopatch.add_feature(feature_type, feature_name, edges[..., np.newaxis])
        #self.debug(mask)
        #print(mask.shape)
        eopatch.add_feature(FeatureType.MASK_TIMELESS, 'LOW_'+feature[1], mask[..., np.newaxis])
        return mask
              
        
    def connected_components(self, image):

        #input_image = eopatch[self.feature_mask[0]][self.feature_mask[1]].squeeze()
        img = image.astype(np.uint8)
        connected = cv2.connectedComponentsWithStats(img)    
        #plt.figure(2,figsize=(50, 50 * aspect_ratio))
        #plt.imshow(color_patches(connected[1]))
        #eopatch.add_feature()
        
        return connected[1]
        
    def execute(self, eopatch):
        eopatch.add_feature(FeatureType.DATA,'B0', eopatch.data['BANDS'][...,[0]])
        eopatch.add_feature(FeatureType.DATA,'B1', eopatch.data['BANDS'][...,[1]])
        eopatch.add_feature(FeatureType.DATA,'B2', eopatch.data['BANDS'][...,[2]])
        print(eopatch)
        bands = eopatch.data['BANDS']
        #print(bands.shape)
        t, w, h, _ = bands.shape
        
        final_mask = np.zeros((w,h))
        
        for arg in self.edge_features:
            edge = self.extract_edges(eopatch, arg['FeatureType'], arg['FeatureName'], arg['YearlyThreshold'],
                                      arg['CannyThresholds'][0],  arg['CannyThresholds'][1], arg['BlurArguments'])
            final_mask = np.logical_and(final_mask, edge)
        eopatch.add_feature(FeatureType.MASK_TIMELESS, 'SUM_EDGES', final_mask[..., np.newaxis])
        
        for unwanted, threshold in self.excluded_features:
            mask = self.filter_unwanted_areas(eopatch,unwanted,threshold, self.dilation_mask, self.erosion_mask)
            final_mask = np.logical_and(final_mask, mask)
            
        image = 1-final_mask
        components = self.connected_components(image)
        
        eopatch.add_feature(self.output_feature[0],self.output_feature[1],components[...,np.newaxis])
        print(eopatch)
        return eopatch


In [33]:

segmentation = Segmentation(
    edge_features = [
                        {   "FeatureType":          FeatureType.DATA,
                            "FeatureName":          'B0',
                            "CannyThresholds":      (20,50),
                            "YearlyThreshold":      0.4,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA,
                            "FeatureName":          'B1',
                            "CannyThresholds":      (40,60),
                            "YearlyThreshold":      0.4,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA,
                            "FeatureName":          'B2',
                            "CannyThresholds":      (30,70),
                            "YearlyThreshold":      0.4,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA,
                            "FeatureName":          'NDVI',
                            "CannyThresholds":      (60,120),
                            "YearlyThreshold":      0.2,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA_TIMELESS,
                            "FeatureName":          'ARGMAX_B4',
                            "CannyThresholds":      (40,80),
                            "YearlyThreshold":      None,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA_TIMELESS,
                            "FeatureName":          'ARGMAX_NDVI',
                            "CannyThresholds":      (20,50),
                            "YearlyThreshold":      None,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA_TIMELESS,
                            "FeatureName":          'ARGMAX_NDVI_SLOPE',
                            "CannyThresholds":      (50,100),
                            "YearlyThreshold":      None,
                            "BlurArguments":        ((5,5),2)
                            },
        
                        {   "FeatureType":          FeatureType.DATA_TIMELESS,
                            "FeatureName":          'ARGMIN_NDVI_SLOPE',
                            "CannyThresholds":      (20,40),
                            "YearlyThreshold":      None,
                            "BlurArguments":        ((9,9),6)
                            }
                    ],
    excluded_features= [((FeatureType.DATA, 'NDVI'), 0.3)],
    dilation_mask = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(7,7)),
    erosion_mask = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)),
    output_feature = (FeatureType.MASK_TIMELESS,'SEGMENTS'))

patch_location = './eopatch/'
load = LoadFromDisk(patch_location)

save_path_location = './eopatch/'
if not os.path.isdir(save_path_location):
    os.makedirs(save_path_location)
    
save = SaveToDisk(save_path_location, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

extra_param ={
    load : {'eopatch_folder': 'patch'},
    save : {'eopatch_folder': 'patch2'}
    }


workflow = LinearWorkflow(
    load,
    segmentation,
    save
    )

workflow.execute(extra_param)  



EOPatch(
  data: {
    B0: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B1: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B2: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B4: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    BANDS: numpy.ndarray(shape=(23, 337, 333, 6), dtype=float64)
    NDVI: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
  }
  mask: {
    EDGES: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    VALID_DATA: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    VEGETATION: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {
    ARGMAX_B4: numpy.ndarray(shape=(337, 333, 1), dtype=int64)
    ARGMAX_NDVI: numpy.ndarray(shape=(337, 333, 1), dtype=int64)
    ARGMAX_NDVI_SLOPE: numpy.ndarray(shape=(337, 333, 1), dtype=uint8)
    ARGMIN_B4: numpy.ndarray(shape=(337, 333, 1), dtype=int64)
    ARGMIN_NDVI: numpy.ndarray(shape=(337, 333, 1), dtype=int6

WorkflowResults(
  Dependency(SaveToDisk): EOPatch(
  data: {
    B0: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B1: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B2: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B4: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    BANDS: numpy.ndarray(shape=(23, 337, 333, 6), dtype=float64)
    NDVI: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
  }
  mask: {
    B0_EDGE: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B1_EDGE: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    B2_EDGE: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    EDGES: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    NDVI_EDGE: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    VALID_DATA: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
    VEGETATION: numpy.ndarray(shape=(23, 337, 333, 1), dtype=float64)
  }
  scalar: {}
  label: {}
  vector: {}
  data_timeless: {
    ARGMAX_